# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

> You can use the classwork notebook and also [my Everything scraping reference](https://jonathansoma.com/everything/scraping)

I shouldn't have to say this, but **do not use ChatGPT for this assignment.**

## Setup: Import what you'll need to scrape the page

We'll be using either Playwrightfor this, *not* requests.

In [198]:
from playwright.async_api import async_playwright
from io import StringIO
from bs4 import BeautifulSoup

import pandas as pd

In [201]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)

In [202]:
# now you can use page to navigate that page
page = await browser.new_page()

In [203]:
await page.goto("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")

<Response url='https://www.tdlr.texas.gov/cimsfo/fosearch.asp' request=<Request url='https://www.tdlr.texas.gov/cimsfo/fosearch.asp' method='GET'>>

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [204]:
await page.locator("text = Search by License Program Type").select_option('Cosmetologists')

['COS']

In [205]:
await page.get_by_label("Search by Last Name").fill("Nguyen")

In [206]:
await page.locator('input[name="B1"]').click()

## Scraping

Once you are on the results page, do this. **I step you through things bit by bit, so it's going to be a little different than we did in class.** Also, no `pd.read_html` allowed because this isn't reaaallly tabular data (or there's just too much to clean up).

Once you've loaded the page: **what selector are you going to do use to find the data?** In class we used things like `.article` or `.head-list`.

> You honestly can do this all with Playwright, no BeautifulSoup involved! But we didn't cover that in class so you should stick to this method:
> 
> ```python
> html = await page.content()
> doc = BeautifulSoup(html)
> ```

In [207]:
html = await page.content()
soup_doc = BeautifulSoup(html)

### Grab the rows, loop through each result and print the entire row

It's probably too many rows to do all at once: once you have a list, use `[:10]` to only show the first ten! For example, if you saved the table rows into `results` you might do something like this:

```python
for result in results[:10]:
    print("------")
    print(result)
```

Although you'd want to print out the text from the row (I give example output below).

In [208]:
all_rows = soup_doc.find_all("tr")

In [224]:
#i know this absolutely is heinous i could not figure out how to get the exact 
# output that you wanted in that format. I wasn't sure if that is the necessity. 
# I know how to get the same output but not in the same format

all_entries = []
for row in all_rows[1:]: # each tr table row
    
    
    #each td in a table row
    one_row = []
    
    
                
    cells = row.find_all("td") # there should be 3 in this list 
       
    td_one = cells[0].find_all("span")
    name = td_one[0].text
    city = td_one[1].text + td_one[2].text
    county = td_one[3].text + td_one[4].text
    zipcode = td_one[5].text + td_one[6].text
    license = td_one[7].text + td_one[8].text
    complaint = td_one[9].text + td_one[10].text
    
    
    td_2 = cells[1].find_all("span")
    date = td_2[0].text + td_2[1].text
    
    desc = cells[1].text + cells[2].text
    
            
    one_row = [name, city, county, zipcode, license, complaint, date, desc]  
    all_entries.append(one_row)
        
    
    
all_entries

[['NGUYEN, ELLIE UYEN ',
  'City:HUMBLE',
  'County:HARRIS',
  'Zip Code:77346',
  ' License #:798636',
  'Complaint #COS20240003068',
  'Date:10/30/2024',
  "Date: 10/30/2024Respondents Ellie Uyen Nguyen and Bao Q. Truong are assessed an administrative penalty in the amount of $3,750.Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean and disinfect all wax pots; Respondents failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone; Respondents failed to dispose of single use items after each use."],
 ['NGUYEN, PHUC ',
  'City:HOUSTON',
  'County:HARRIS',
  'Zip Code:77083',
  ' License #:807327',
  'Complaint #COS20

In [225]:
for entry in all_entries[:10]:
    print("----")
    
    for data in entry:
        print(data)

----
NGUYEN, ELLIE UYEN 
City:HUMBLE
County:HARRIS
Zip Code:77346
 License #:798636
Complaint #COS20240003068
Date:10/30/2024
Date: 10/30/2024Respondents Ellie Uyen Nguyen and Bao Q. Truong are assessed an administrative penalty in the amount of $3,750.Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean and disinfect all wax pots; Respondents failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone; Respondents failed to dispose of single use items after each use.
----
NGUYEN, PHUC 
City:HOUSTON
County:HARRIS
Zip Code:77083
 License #:807327
Complaint #COS20230005653
Date:10/9/2024
Date: 10/9/2024Respondent is asses

The result should look something like this:

```
Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, LONG D
City: SAN SABA
County: SAN SABA
Zip Code: 76877
```

### Loop through each result and print each person's name

You'll probably get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything when there's an error, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

Output should look like this:

```
Doesn't have a name
NGUYEN, THANH
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
```

* *Tip: The name has a class you can use. The class name is reused in a lot of places, but because it's the first one you don't have to worry about that!*
* *Tip: Instead of searching across the entire page – like `doc.select_one` – you should be doing your searching just inside of each **row** (I used this technique in the beginning of class with BeautifulSoup when we were scraping the books page)* 

In [229]:
for row in all_rows:
    
    try:
        
        print(row.find(class_ = "results_text").text)
       # try to do something
    
    except:
       print("It didn't work")

It didn't work
NGUYEN, ELLIE UYEN 
NGUYEN, PHUC 
NGUYEN, TRONG 
NGUYEN, SA 
NGUYEN, LYNH 
NGUYEN, LISA 
NGUYEN, CHAU MINH 
NGUYEN, KEVIN C
NGUYEN, THANH D
NGUYEN, TAM THI 
NGUYEN, DUNG THANH 
NGUYEN, AJ 
NGUYEN, LAM THE 
NGUYEN, QUY KIM 
NGUYEN, THI THIBICH 
NGUYEN, TONY MINH 
NGUYEN, JIMMY 
NGUYEN, CECILIA L
NGUYEN, JENNIFER HUE 
NGUYEN, ANH DUC PHONG 
NGUYEN, THAO THANH 
NGUYEN, JOHN T
NGUYEN, THUY THU THI 
NGUYEN, STEVEN T
NGUYEN, DANIEL THAO 
NGUYEN, THI THUY 
NGUYEN, JOHN 
NGUYEN, STEVEN QUOC 
NGUYEN, THU THI 
NGUYEN, TUONG 
NGUYEN, HUONG T
NGUYEN, STEVEN QUOC 
NGUYEN, HUONG THI NGOC 
NGUYEN, TAMMY 
NGUYEN, TOAN DUC 
NGUYEN, NHAN TRONG 
NGUYEN, TAM PHUONG 
NGUYEN, NHUNG NGOC THI 
NGUYEN, ANH NGOC 
NGUYEN, HUYEN THI 
NGUYEN, DANIEL 
NGUYEN, BANG 
NGUYEN, HONG N
NGUYEN, TUYET 
NGUYEN, TUYET MINH 
NGUYEN, JOHN THANH NHAN 
NGUYEN, THE THI 
NGUYEN, THAO HONG 
NGUYEN, NGOC THI 
NGUYEN, TUYEN THANH 
NGUYEN, MINH THU 
NGUYEN, LOAN THI NGOC 
NGUYEN, JACK 
NGUYEN, HOANG DRAGON 
NGUYEN, KELL

## Loop through each result, printing each violation description ("Basis for order")

Your results should look something like:

```
Doesn't have a violation
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
...
```

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: Or I guess you could just skip the one with the problem using try/except...*

In [234]:
for row in all_rows:
    
    try:
        
        cells = row.find_all("td")
    
        print(cells[2].text)
        print("\n")
        
    except:
        
        print("did not work")
        print("\n")
    

did not work


Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean and disinfect all wax pots; Respondents failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone; Respondents failed to dispose of single use items after each use.


Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required daily, bi-weekly, and before use by each patron.


Respondents failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respond

## Loop through each result, printing the complaint number

Output should look similar to this:

```
Doesn't have a complaint number
COS20210004784
COS20210009745
COS20210011484
...
```

- *Tip: Think about the order of the elements. Can you count from the opposite direction than you normally do?*

In [237]:
for row in all_rows:
    
    try:
        
        cells = row.find_all("td")[0].find_all("span")
    
        print(cells[-1].text)
        
    except:
        
        print("did not work")

did not work
COS20240003068
COS20230005653
COS20240011518
COS20240008640
COS20230004635
COS20230010686
COS20220007736
COS20230020006
COS20240000594
COS20240008070
COS20240001947
COS20230009966
COS20220016043
COS20230015854
COS20220002410
COS20230002569
COS20230013709
COS20230020593
COS20240005064
COS20230018810
COS20230000401
COS20220009661
COS20230013043
COS20230001042
COS20230002419
COS20240002426
COS20220015806
COS20230007454
COS20230009980
COS20230001960
COS20220017901
COS20220018135
COS20220016107
COS20220013443
COS20220015714
COS20220012646
COS20220012645
COS20230010084
COS20230010404
COS20220015853
COS20220008852
COS20230009558
COS20230010492
COS20230000043
COS20220006122
COS20220009835
COS20210007272
COS20210012244
COS20210016437
COS20220007001
COS20220009623
COS20220014041
COS20220016162
COS20210009264
COS20210012950
COS20210013902
COS20220005426
COS20210012469
COS20220004591
COS20220005795
COS20230000013
COS20220000293
COS20210012920
COS20220006428
COS20220011063
COS202300026

## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

Based on what you print out, the output might look something like:

```
This row is broken: Name and Location Order Basis for Order
{'name': 'NGUYEN, THANH', 'city': 'FRISCO', 'county': 'COLLIN', 'zip_code': '75034', 'complaint_no': 'COS20210004784', 'license_numbers': '790672', 'complaint': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.'}
{'name': 'NGUYEN, LONG D', 'city': 'SAN SABA', 'county': 'SAN SABA', 'zip_code': '76877', 'complaint_no': 'COS20210009745', 'license_numbers': '760420, 1620583', 'complaint': 'Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.'}
```

> _**Tip:** Depending on how you do this, you might want to figure out how to search for "next siblings" or "following siblings"_

In [244]:
# I excluded the header manually so didn't use a try and except block
no_header = all_rows[1:]

final_results =[]

for row in no_header:
    
    one_row = {}
    one_row["name"] = row.find(class_ = "results_text").text
    one_row["city"] = row.find_all("td")[0].find_all("span")[2].text
    one_row["county"] = row.find_all("td")[0].find_all("span")[4].text
    one_row["zipcode"] = row.find_all("td")[0].find_all("span")[6].text
    one_row["complaint_no"] = row.find_all("td")[0].find_all("span")[-1].text
    one_row["license_numbers"] = row.find_all("td")[0].find_all("span")[8].text
    one_row["complaint"] = row.find_all("td")[2].text
    
    final_results.append(one_row)

In [245]:
final_results[:2]

[{'name': 'NGUYEN, ELLIE UYEN ',
  'city': 'HUMBLE',
  'county': 'HARRIS',
  'zipcode': '77346',
  'complaint_no': 'COS20240003068',
  'license_numbers': '798636',
  'complaint': "Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean and disinfect all wax pots; Respondents failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone; Respondents failed to dispose of single use items after each use."},
 {'name': 'NGUYEN, PHUC ',
  'city': 'HOUSTON',
  'county': 'HARRIS',
  'zipcode': '77083',
  'complaint_no': 'COS20230005653',
  'license_numbers': '807327',
  'complaint': 'Respondent failed to follow whirlpool foot spas c

### Save that to a CSV named `output.csv`

The dataframe should look something like...

|index|name|city|county|zip_code|complaint_no|license_numbers|complaint|
|---|---|---|---|---|---|---|---|
|0|NGUYEN, THANH|FRISCO|COLLIN|75034|COS20210004784|790672|Respondent failed to clean and sanitize whirlp...|
|1|NGUYEN, LONG D|SAN SABA|SAN SABA|76877|COS20210009745|760420, 1620583|Respondent failed to keep a record of the date...|


- *Tip: If you send a list of dictionaries to `pd.DataFrame(...)`, it will create a dataframe out of that list!*

In [247]:
filename = "output.csv"
df = pd.json_normalize(final_results)
df.to_csv(filename, index = False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [248]:
file = pd.read_csv("/Users/anushasubramanian/Desktop/08-homework/output.csv")
df = pd.DataFrame(file)
df.head()

,name,city,county,zipcode,complaint_no,license_numbers,complaint
0,"NGUYEN, ELLIE UYEN",HUMBLE,HARRIS,77346,COS20240003068,798636,"Respondents failed to clean, disinfect, and st..."
1,"NGUYEN, PHUC",HOUSTON,HARRIS,77083,COS20230005653,807327,Respondent failed to follow whirlpool foot spa...
2,"NGUYEN, TRONG",FORT WORTH,TARRANT,76177,COS20240011518,808004,Respondents failed to follow whirlpool foot sp...
3,"NGUYEN, SA",AUSTIN,TRAVIS,78729,COS20240008640,799667,Respondent failed to follow whirlpool foot spa...
4,"NGUYEN, LYNH",RICHARDSON,DALLAS,75081,COS20230004635,811783,Respondent failed to follow whirlpool foot spa...
